# Downloading products from GEE into netcdfs


## Load packages

In [ ]:
import os
import ee
import wxee
import warnings
import xarray as xr
import pandas as pd
import numpy as np
from odc.geo.xr import assign_crs

## Authenticate GEE    

In [ ]:
# Trigger the authentication flow.
# ee.Authenticate()

# Initialize the library.
ee.Initialize()

## Analysis parameters


In [ ]:
product = 'MODIS/MCD43A4_006_NDVI'
bands = ['NDVI']
start_date = '1/1/2001'
end_date = '12/31/2022'
output_crs = "EPSG:3577"
resolution = 5000

# ~Aus region (slightly less to trick google into giving me 1km res)
region = ee.Geometry.Polygon([[
            [114,-43.0],
            [154.0,-43.0],
            [154.0,-10.0],
            [113,-10.0],
            [113,-43.0]]])


In [ ]:
#use start and end dates to loop through months and load GEE data
start = pd.date_range(start=start_date, end=end_date, freq='MS') 
end = pd.date_range(start=start_date, end=end_date, freq='M')

i = 0
for s, e in zip(start,end):
    print(" {:03}/{:03}\r".format(i+1, len(start)), end="")

    ss = s+pd.Timedelta(14, 'd')

    s = s.strftime('%Y-%m-%d')
    e = e.strftime('%Y-%m-%d')

    ds = ee.ImageCollection(product) \
      .filterDate(s,e) \
      .map(lambda image: image.clip(region)) \
      .select(bands) \
      .reduce(ee.Reducer.median())  
      
    export = ee.batch.Export.image.toDrive(ds,
        description='MODIS_NDVI_median'+'_'+ss.strftime('%Y-%m-%d'),
        folder='MODIS_NDVI_median',
        region=region,
        scale=resolution,
        crs=output_crs,
        maxPixels=1e13,
        fileFormat='GeoTIFF',
        formatOptions={
            'cloudOptimized':True
              }
        )
    export.start()
    break
    i+=1

In [ ]:
# %%time
# #use start and end dates to loop through months and load GEE data

# start = pd.date_range(start=start_date, end=end_date, freq='MS') 
# end = pd.date_range(start=start_date, end=end_date, freq='M')

# i = 0
# for s, e in zip(start,end):
#     print(" {:03}/{:03}\r".format(i+1, len(start)), end="")

#     ss = s+pd.Timedelta(14, 'd')

#     if os.path.isfile(output_path+'MODIS_NDVI_'+ss.strftime('%Y-%m')+'.nc'):
#         i+=1
#         pass

#     else:
#         s = s.strftime('%Y-%m-%d')
#         e = e.strftime('%Y-%m-%d')

#         ds = ee.ImageCollection(product) \
#           .filterDate(s,e) \
#           .map(lambda image: image.clip(region))

#         ds = ds.select(bands)

#         ts = wxee.TimeSeries(ds)
#         ds = ts.wx.to_xarray(region=region, scale=resolution, crs=output_crs, progress=False, num_cores=4)

#         warnings.filterwarnings("ignore")
#         ds = ds.astype(np.float32)

#         ds_mean = ds.resample(time='MS', loffset=pd.Timedelta(14, 'd')).mean()

#         ds_mean = ds_mean.rename({'NDVI':'NDVI_mean'})
#         ds_mean.attrs['nodata'] = np.nan

#         ds_out = assign_crs(ds_mean, crs='epsg:3577')

#         ds_out.to_netcdf(output_path+'MODIS_NDVI_'+ss.strftime('%Y-%m')+'.nc')

#         i+=1
